## การฝึกโมเดล CBoW

สมุดบันทึกนี้เป็นส่วนหนึ่งของ [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

ในตัวอย่างนี้ เราจะมาดูการฝึกโมเดลภาษาประเภท CBoW เพื่อสร้างพื้นที่เวกเตอร์ Word2Vec ของเราเอง โดยเราจะใช้ชุดข้อมูล AG News เป็นแหล่งข้อมูลข้อความ


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ก่อนอื่นเรามาโหลดชุดข้อมูลของเราและกำหนดตัวแยกคำและคำศัพท์ เราจะตั้งค่า `vocab_size` เป็น 5000 เพื่อจำกัดการคำนวณเล็กน้อย


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## โมเดล CBoW

CBoW เรียนรู้การทำนายคำโดยใช้คำที่อยู่ใกล้เคียง $2N$ คำเป็นข้อมูลนำเข้า ตัวอย่างเช่น เมื่อ $N=1$ เราจะได้คู่คำต่อไปนี้จากประโยค *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks) โดยคำแรกคือคำที่อยู่ใกล้เคียงที่ใช้เป็นข้อมูลนำเข้า และคำที่สองคือคำที่เราต้องการทำนาย

ในการสร้างเครือข่ายเพื่อทำนายคำถัดไป เราจะต้องป้อนคำที่อยู่ใกล้เคียงเป็นข้อมูลนำเข้า และได้หมายเลขคำเป็นผลลัพธ์ สถาปัตยกรรมของเครือข่าย CBoW มีดังนี้:

* คำที่ป้อนเข้าจะถูกส่งผ่านชั้นฝังตัว (embedding layer) ชั้นฝังตัวนี้จะเป็น Word2Vec embedding ของเรา ดังนั้นเราจะกำหนดมันแยกต่างหากเป็นตัวแปร `embedder` ในตัวอย่างนี้เราจะใช้ขนาด embedding = 30 แม้ว่าคุณอาจต้องการทดลองกับมิติที่สูงขึ้น (Word2Vec จริงมี 300)
* เวกเตอร์ embedding จะถูกส่งต่อไปยังชั้นเชิงเส้น (linear layer) ที่จะทำนายคำผลลัพธ์ ดังนั้นมันจะมีจำนวนเซลล์ประสาทเท่ากับ `vocab_size`

สำหรับผลลัพธ์ หากเราใช้ `CrossEntropyLoss` เป็นฟังก์ชันการสูญเสีย เราจะต้องให้หมายเลขคำเป็นผลลัพธ์ที่คาดหวัง โดยไม่ต้องใช้การเข้ารหัสแบบ one-hot


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## การเตรียมข้อมูลสำหรับการฝึก

ตอนนี้เรามาเขียนฟังก์ชันหลักที่จะสร้างคู่คำ CBoW จากข้อความกัน ฟังก์ชันนี้จะช่วยให้เรากำหนดขนาดของหน้าต่างได้ และจะคืนชุดของคู่คำ - คำที่เป็นข้อมูลนำเข้าและคำที่เป็นผลลัพธ์ โปรดทราบว่าฟังก์ชันนี้สามารถใช้งานได้ทั้งกับคำและกับเวกเตอร์/เทนเซอร์ ซึ่งจะช่วยให้เราสามารถเข้ารหัสข้อความก่อนที่จะส่งไปยังฟังก์ชัน `to_cbow`


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


เรามาเตรียมชุดข้อมูลการฝึกกันเถอะ เราจะตรวจสอบข่าวทั้งหมด เรียกใช้ `to_cbow` เพื่อรับรายการคู่คำ และเพิ่มคู่คำเหล่านั้นลงใน `X` และ `Y` เพื่อประหยัดเวลา เราจะพิจารณาเฉพาะข่าว 10,000 รายการแรกเท่านั้น - คุณสามารถลบข้อจำกัดนี้ได้หากคุณมีเวลารอมากขึ้น และต้องการได้เวกเตอร์ฝังตัวที่ดียิ่งขึ้น :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

เราจะเปลี่ยนข้อมูลนั้นให้เป็นชุดข้อมูลเดียว และสร้างตัวโหลดข้อมูล:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

เราจะเปลี่ยนข้อมูลนั้นให้เป็นชุดข้อมูลเดียว และสร้างตัวโหลดข้อมูล:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

ตอนนี้เรามาเริ่มการฝึกจริงกัน เราจะใช้ตัวปรับแต่ง `SGD` พร้อมกับอัตราการเรียนรู้ที่ค่อนข้างสูง คุณยังสามารถลองใช้ตัวปรับแต่งอื่นๆ เช่น `Adam` ได้อีกด้วย เราจะฝึกเป็นเวลา 10 รอบในตอนเริ่มต้น - และคุณสามารถรันเซลล์นี้ใหม่ได้หากต้องการลดค่าความสูญเสียให้ต่ำลง


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## ทดลองใช้ Word2Vec

เพื่อใช้งาน Word2Vec เรามาเริ่มดึงเวกเตอร์ที่สอดคล้องกับคำทั้งหมดในคลังคำศัพท์ของเรากัน:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


น่าสนใจที่จะใช้ Word2Vec เพื่อค้นหาคำพ้องความหมาย ฟังก์ชันต่อไปนี้จะคืนค่าคำที่ใกล้เคียงที่สุด `n` คำสำหรับอินพุตที่กำหนด ในการค้นหาคำเหล่านั้น เราคำนวณนอร์มของ $|w_i - v|$ โดยที่ $v$ คือเวกเตอร์ที่สอดคล้องกับคำอินพุตของเรา และ $w_i$ คือการเข้ารหัสของคำที่ $i$ ในคลังคำศัพท์ จากนั้นเราจะจัดเรียงอาร์เรย์และคืนค่าดัชนีที่สอดคล้องกันโดยใช้ `argsort` และนำองค์ประกอบ `n` แรกของรายการ ซึ่งเข้ารหัสตำแหน่งของคำที่ใกล้เคียงที่สุดในคลังคำศัพท์


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## ข้อคิดสำคัญ

การใช้เทคนิคที่ชาญฉลาดอย่าง CBoW เราสามารถฝึกโมเดล Word2Vec ได้ คุณยังสามารถลองฝึกโมเดล skip-gram ซึ่งถูกออกแบบมาเพื่อทำนายคำที่อยู่รอบๆ โดยใช้คำตรงกลาง และดูว่ามันทำงานได้ดีแค่ไหน



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์ที่เป็นมืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
